In [1]:
DATA= 'dicom/MICCAI_BraTS_2019_Data_Training'
DATA_HGG = DATA +'/HGG/'
DATA_LGG = DATA +'/LGG/'

FLAIR = 'flair'
T1 = 't1'
T2 = 't2'
T1CE = 't1ce'

In [2]:
import os, sys, glob
import numpy as np
import SimpleITK as sitk
import sys
import os
import pandas as pd
import re

In [3]:
import pandas
import numpy

# Data Preprocessing

## Run N4BiasFieldCorrectionImageFilter

In [4]:

def n4cor(in_img, out_img):
    inputImage = sitk.ReadImage(in_img )
    maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

    inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter();
    numberFilltingLevels = 4

    output = corrector.Execute( inputImage, maskImage ) 
    sitk.WriteImage( output, out_img )

## find all relevant image directories

In [5]:
input_path = DATA_HGG
        
hgg_paths = []
for dirpath, dirnames, files in os.walk(input_path):
    for dir in dirnames:
        if ('BraTS19' in dir):
            hgg_paths.append(dir)

input_path = DATA_LGG
        
lgg_paths = []
for dirpath, dirnames, files in os.walk(input_path):
    for dir in dirnames:
        if ('BraTS19' in dir):
            lgg_paths.append(dir)

In [6]:
len(lgg_paths),len(hgg_paths)

(76, 259)

## Run n4corr filter on T1 and T1CE images

In [7]:
%%time

for dirpath, dirnames, files in os.walk(input_path):

    for file in files:
        if ('_t1.' in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath, file[:-9]+ 't1_nb4.nii.gz')
            print(in_img, out_img)
            if (not os.path.isfile(out_img)):
                n4cor(in_img, out_img)
        
        if (T1CE in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath,file[:-11]+ 't1ce_nb4.nii.gz')
            print(in_img, out_img)
            if (not os.path.isfile(out_img)):
                n4cor(in_img, out_img)

In [8]:
VALIDATION_DATA = 'dicom/MICCAI_BraTS_2019_Data_Validation/'

In [9]:
%%time

for dirpath, dirnames, files in os.walk(VALIDATION_DATA):

    for file in files:
        if ('_t1.' in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath, file[:-9]+ 't1_nb4.nii.gz')
            if (not os.path.isfile(out_img)):
                print(in_img, out_img)
                n4cor(in_img, out_img)
        
        if (T1CE in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath,file[:-11]+ 't1ce_nb4.nii.gz')
            if (not os.path.isfile(out_img)):
                print(in_img, out_img)
                n4cor(in_img, out_img)

dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W047_1/BraTS19_WashU_W047_1_t1.nii.gz dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W047_1/BraTS19_WashU_W047_1_t1_nb4.nii.gz
dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W047_1/BraTS19_WashU_W047_1_t1ce.nii.gz dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W047_1/BraTS19_WashU_W047_1_t1ce_nb4.nii.gz
dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W033_1/BraTS19_WashU_W033_1_t1ce.nii.gz dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W033_1/BraTS19_WashU_W033_1_t1ce_nb4.nii.gz
dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W033_1/BraTS19_WashU_W033_1_t1.nii.gz dicom/MICCAI_BraTS_2019_Data_Validation/BraTS19_WashU_W033_1/BraTS19_WashU_W033_1_t1_nb4.nii.gz
CPU times: user 1h 11min 28s, sys: 42.5 s, total: 1h 12min 10s
Wall time: 25min 19s
